# CS 20 : TensorFlow for Deep Learning Research
## Lecture 11 : Recurrent Neural Networks
Simple example for Sequence to Sequence Classification (Simple pos tagger) by Stacked Bi-directional GRU with Drop out. 

### Sequence to Sequence Classification by Stacked Bi-directional GRU with Drop out
- Creating the **data pipeline** with `tf.data`
- Preprocessing word sequences (variable input sequence length) by `padding technique`
- Creating `user function (make batch)` for preprocessing word sequences
- Training **many to many classification** with `tf.contrib.seq2seq.sequence_loss`
- Masking unvalid token with `tf.sequence_mask`
- Applying **Drop out** to model by `tf.contrib.rnn.DropoutWrapper`
- Applying **Stacking** and **dynamic rnn** to model by `tf.contrib.rnn.stack_bidirectional_dynamic_rnn`
- Creating the model as **Class**
- Reference
    - https://github.com/aisolab/sample_code_of_Deep_learning_Basics/blob/master/DLEL/DLEL_12_2_RNN_(toy_example).ipynb

### Setup

In [1]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import string
%matplotlib inline

slim = tf.contrib.slim
print(tf.__version__)

1.8.0


### Prepare example data 

In [2]:
sentences = [['I', 'feel', 'hungry'],
     ['tensorflow', 'is', 'very', 'difficult'],
     ['tensorflow', 'is', 'a', 'framework', 'for', 'deep', 'learning'],
     ['tensorflow', 'is', 'very', 'fast', 'changing']]
pos = [['pronoun', 'verb', 'adjective'],
     ['noun', 'verb', 'adverb', 'adjective'],
     ['noun', 'verb', 'determiner', 'noun', 'preposition', 'adjective', 'noun'],
     ['noun', 'verb', 'adverb', 'adjective', 'verb']]

In [3]:
# word dic
word_list = []
for elm in sentences:
    word_list += elm
word_list = list(set(word_list))
word_list.sort()
word_list = ['pad'] + word_list

word_dic = {word : idx for idx, word in enumerate(word_list)}
print(word_dic)

{'pad': 0, 'I': 1, 'a': 2, 'changing': 3, 'deep': 4, 'difficult': 5, 'fast': 6, 'feel': 7, 'for': 8, 'framework': 9, 'hungry': 10, 'is': 11, 'learning': 12, 'tensorflow': 13, 'very': 14}


In [4]:
# pos dic
pos_list = []
for elm in pos:
    pos_list += elm
pos_list = list(set(pos_list))
pos_list.sort()
pos_list = ['pad'] + pos_list
print(pos_list)

pos_dic = {pos : idx for idx, pos in enumerate(pos_list)}
pos_dic

['pad', 'adjective', 'adverb', 'determiner', 'noun', 'preposition', 'pronoun', 'verb']


{'pad': 0,
 'adjective': 1,
 'adverb': 2,
 'determiner': 3,
 'noun': 4,
 'preposition': 5,
 'pronoun': 6,
 'verb': 7}

In [5]:
pos_idx_to_dic = {elm[1] : elm[0] for elm in pos_dic.items()}
pos_idx_to_dic

{0: 'pad',
 1: 'adjective',
 2: 'adverb',
 3: 'determiner',
 4: 'noun',
 5: 'preposition',
 6: 'pronoun',
 7: 'verb'}

### Create make_batch function

In [6]:
def make_batch(sequences, max_len, dic):
    seq_len = []
    seq_batch = []
    for seq in sequences:
        seq_len.append(len(seq))
        seq_idx = [dic.get(word) for word in seq]
        seq_idx += (max_len - len(seq_idx)) * [dic.get('pad')] # 14 is idx of meaningless token "*"
        seq_matrix = np.eye(len(dic))[seq_idx].tolist()
        seq_batch.append(seq_matrix)        
    return seq_len, seq_batch

### Pre-process data

In [7]:
maxlen = 10
X_length, X_batch = make_batch(sequences = sentences, max_len = maxlen, dic = word_dic)
print(X_length, np.shape(X_batch))

[3, 4, 7, 5] (4, 10, 15)


In [8]:
y = [elm + ['pad'] * (maxlen - len(elm)) for elm in pos]
y = [list(map(lambda el : pos_dic.get(el), elm)) for elm in y]
print(np.shape(y))

(4, 10)


In [9]:
y

[[6, 7, 1, 0, 0, 0, 0, 0, 0, 0],
 [4, 7, 2, 1, 0, 0, 0, 0, 0, 0],
 [4, 7, 3, 4, 5, 1, 4, 0, 0, 0],
 [4, 7, 2, 1, 7, 0, 0, 0, 0, 0]]

### Define SimPosStackedBiGRU

In [10]:
class SimPosStackedBiGRU:
    def __init__(self, X_length, X_batch, y, n_of_classes, hidden_dims, max_len):
        
        # Data pipeline
        with tf.variable_scope('input_layer'):
            self._X_length = X_length
            self._X_batch = X_batch
            self._y = y
            self._keep_prob = tf.placeholder(dtype = tf.float32)
            
        # Stacked Bi-directional GRU (many to many)
        with tf.variable_scope('stacked_bi-directional_gru'):
            
            # forward 
            gru_fw_cells = []
            for hidden_dim in hidden_dims:
                gru_fw_cell = tf.contrib.rnn.GRUCell(num_units = hidden_dim, activation = tf.nn.tanh)
                gru_fw_cell = tf.contrib.rnn.DropoutWrapper(cell = gru_fw_cell,
                                                            output_keep_prob = self._keep_prob)
                gru_fw_cells.append(gru_fw_cell)
            
            # backword
            gru_bw_cells = []
            for hidden_dim in hidden_dims:
                gru_bw_cell = tf.contrib.rnn.GRUCell(num_units = hidden_dim, activation = tf.nn.tanh)
                gru_bw_cell = tf.contrib.rnn.DropoutWrapper(cell = gru_bw_cell,
                                                            output_keep_prob = self._keep_prob)
                gru_bw_cells.append(gru_bw_cell)
                            
            outputs, _, _ = tf.contrib.rnn.stack_bidirectional_dynamic_rnn(cells_fw = gru_fw_cells, 
                                                                           cells_bw = gru_bw_cells,
                                                                           inputs = self._X_batch,
                                                                           sequence_length = self._X_length,
                                                                           dtype = tf.float32)
            
            weights = tf.get_variable(name = 'weights', shape = (2 * hidden_dim, n_of_classes),
                                      initializer = tf.contrib.layers.xavier_initializer())
            
            self._score = tf.map_fn(lambda elm : tf.matmul(elm, weights), outputs)

        with tf.variable_scope('seq2seq_loss'):
            masks = tf.sequence_mask(lengths = self._X_length, maxlen = max_len, dtype = tf.float32)
            self.seq2seq_loss = tf.contrib.seq2seq.sequence_loss(logits = self._score, targets = self._y,
                                                                 weights = masks)
    
        with tf.variable_scope('prediction'):
            self._prediction = tf.argmax(input = self._score,
                                         axis = 2, output_type = tf.int32)
    
    def predict(self, sess, X_length, X_batch, keep_prob = 1.):
        feed_prediction = {self._X_length : X_length, self._X_batch : X_batch, self._keep_prob : keep_prob}
        return sess.run(self._prediction, feed_dict = feed_prediction)

### Create a model of SimPosStackedBiGRU

In [11]:
# hyper-parameter#
lr = .003
epochs = 100
batch_size = 2
total_step = int(np.shape(X_batch)[0] / batch_size)
print(total_step)

2


In [12]:
## create data pipeline with tf.data
tr_dataset = tf.data.Dataset.from_tensor_slices((X_length, X_batch, y))
tr_dataset = tr_dataset.shuffle(buffer_size = 20)
tr_dataset = tr_dataset.batch(batch_size = batch_size)
tr_iterator = tr_dataset.make_initializable_iterator()
print(tr_dataset)

<BatchDataset shapes: ((?,), (?, 10, 15), (?, 10)), types: (tf.int32, tf.float32, tf.int32)>


In [13]:
X_length_mb, X_batch_mb, y_mb = tr_iterator.get_next()

In [14]:
sim_pos_stacked_bi_gru = SimPosStackedBiGRU(X_length = X_length_mb, X_batch = X_batch_mb, y = y_mb,
                                            n_of_classes = 8, hidden_dims = [16,16], max_len = maxlen)

### Creat training op and train model

In [15]:
## create training op
opt = tf.train.AdamOptimizer(learning_rate = lr)
training_op = opt.minimize(loss = sim_pos_stacked_bi_gru.seq2seq_loss)

In [16]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

tr_loss_hist = []

for epoch in range(epochs):
    avg_tr_loss = 0
    tr_step = 0
    
    sess.run(tr_iterator.initializer)
    try:
        while True:
            _, tr_loss = sess.run(fetches = [training_op, sim_pos_stacked_bi_gru.seq2seq_loss],
                                  feed_dict = {sim_pos_stacked_bi_gru._keep_prob : .5})
            avg_tr_loss += tr_loss
            tr_step += 1
            
    except tf.errors.OutOfRangeError:
        pass
    
    avg_tr_loss /= tr_step
    tr_loss_hist.append(avg_tr_loss)
    if (epoch + 1) % 10 == 0:
        print('epoch : {:3}, tr_loss : {:.3f}'.format(epoch + 1, avg_tr_loss))

epoch :  10, tr_loss : 1.909
epoch :  20, tr_loss : 1.678
epoch :  30, tr_loss : 1.513
epoch :  40, tr_loss : 1.499
epoch :  50, tr_loss : 1.016
epoch :  60, tr_loss : 1.321
epoch :  70, tr_loss : 1.210
epoch :  80, tr_loss : 0.847
epoch :  90, tr_loss : 0.863
epoch : 100, tr_loss : 0.768


In [17]:
yhat = sim_pos_stacked_bi_gru.predict(sess = sess, X_length = X_length, X_batch = X_batch)
yhat

array([[6, 7, 1, 0, 0, 0, 0, 0, 0, 0],
       [4, 7, 2, 1, 0, 0, 0, 0, 0, 0],
       [4, 7, 3, 4, 5, 1, 4, 0, 0, 0],
       [4, 7, 2, 1, 1, 0, 0, 0, 0, 0]], dtype=int32)

In [18]:
y

[[6, 7, 1, 0, 0, 0, 0, 0, 0, 0],
 [4, 7, 2, 1, 0, 0, 0, 0, 0, 0],
 [4, 7, 3, 4, 5, 1, 4, 0, 0, 0],
 [4, 7, 2, 1, 7, 0, 0, 0, 0, 0]]

In [19]:
yhat = [list(map(lambda elm : pos_idx_to_dic.get(elm), row)) for row in yhat]
for elm in yhat:
    print(elm)

['pronoun', 'verb', 'adjective', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad']
['noun', 'verb', 'adverb', 'adjective', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad']
['noun', 'verb', 'determiner', 'noun', 'preposition', 'adjective', 'noun', 'pad', 'pad', 'pad']
['noun', 'verb', 'adverb', 'adjective', 'adjective', 'pad', 'pad', 'pad', 'pad', 'pad']
